In [ ]:
! pip install sentence-transformers
! pip install qdrant_client

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from sentence_transformers import SentenceTransformer


In [ ]:
loaders = [
    PyPDFLoader("Medical-Encyclopedia.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
chunk_size=2500
chunk_overlap = 250
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

splits = r_splitter.split_documents(docs)
# splits[0]
chunks = [doc.page_content for doc in splits]

In [ ]:
model = SentenceTransformer("neuml/pubmedbert-base-embeddings")
vectors = model.encode(chunks)

In [ ]:
chunks[0]

In [ ]:
len(vectors[7])


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")

In [ ]:
client.recreate_collection(
    collection_name="medical-encyclopedia",
    vectors_config=VectorParams(size=len(vectors[0]), distance=Distance.COSINE),
)

In [ ]:
chunks = [doc.page_content for doc in splits]
client.upload_collection(
    collection_name="medical-encyclopedia",
    ids=[i for i in range(len(chunks))],
    vectors=vectors,
)

In [ ]:
question = "Tell me about cartilage?"

In [ ]:
def make_context(question):
  ques_vector = model.encode(question)
  result = client.query_points(
      collection_name="medical-encyclopedia",
      query=ques_vector,


  )

  sim_ids = []
  for i in result.points:
    sim_ids.append(i.id)
  context = ""
  for i in sim_ids[0:2]:
    context+=chunks[i]
  return context

In [ ]:
from groq import Groq

groq_api = Groq(api_key="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")
def respond(question):
    chat_completion = groq_api.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"This is the question asked by user {question} and the context given is {make_context(question)} answer this question based on the context provided",
            }
                ],
                model="llama-3.1-70b-versatile",
            )
    return chat_completion.choices[0].message.content

In [ ]:
#print(make_context(question))

respond(question)

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(
        """
        <div style="text-align: center;">
            <h1 style="color: #0078D7;">Healthcare Chatbot</h1>
        </div>
        """
    )

    gr.Markdown(
        """
        <p style="text-align: center; font-size: 18px; color: #555;">
            Hello! Just ask me anything about any disease.
        </p>
        """
    )

    gr.Markdown("<hr/>")

    user_query = gr.Textbox(placeholder="E.g., Tell me about cartilage?", label="Enter your question:")
    
    output = gr.HTML()

    def generate_response(user_query):
        bot_response = respond(user_query)
        return f"""
        <div style="background-color: #f9f9f9; padding: 10px; border-radius: 5px; margin-top: 20px;">
            <h4 style="color: #0078D7;">Answer:</h4>
            <p style="color: #333;">{bot_response}</p>
        </div>
        """

    submit_button = gr.Button("Answer")
    submit_button.click(fn=generate_response, inputs=user_query, outputs=output)

demo.launch()
